# PyTorch Computer Vision -art of teaching a computer to see.

Typical inputs and outputs of CV model:

INPUT: images wih 3 channels : width, height and color channels(RGB)

-> CONVERT IT INTO TENSORs

-> goes into ML model (CNN)

-> OUTPUT: predicted probabilities

OUPUT: predcited class the image belongs to


`shape= [batch_size, width, height,colour_channels]` -> **NHWC** => *colour channels last*

`shape= [batch_size,colour_channels  width, height,]` -> **NCHW** => *colour chanels first*


# What is a Convolutional Neural Network (CNN)?

+ A convolutional neural network is a specific kind of neural network with multiple layers. It processes data that has a grid-like arrangement then extracts important features.


+ CNNs use convolutions to handle the math behind the scenes. A convolution is used instead of matrix multiplication in at least one layer of the CNN

+ `Convolutions` :a *mathematical operation* performed on two functions that yields a function that is a combination of the two original functions.

  The convolution operation follows the properties of multiplication:

  1.   Commutative: `fg = gf`
  2.  Associative: `(fg)h = f(gh)`
  3. Distributive: `h(f+g) = hf + hg`


#Typical Architecture of a CNN

Hyperparameters or layer type


1.  **Input images**
2.   **Input layer**  -> takes images and preprocess it `input_shape`
3. **Convolution Layer** -> Extracts and learns the most important features from target images `torch.nn.ConvXd()`
4. **hidden activation** ->Adds non-linearity to learned features `torch.nn.ReLU()`
5.**Pooling layer** -> Reduces the dimensionality of learned image features Max:`torch.nn.MaxPool2d()` Avg: `torch.nn.AvgPool2d()`
6. **Output/linear layer** -> `torch.nn.Linear(out_features=[no_of_classes])`
7. **ouput activation** -> converts logits to predictions prob `torch.sigmoid()` or `torch.softmax()`



---


CV libraries:
`torchvision`

`torchvision.datasets`

`torchvision.models` => Transfer learning

`torchvision.transforms`

`torch.utils.data.Dataset`

`torch.utils.data.DataLoader`

---






In [1]:
import torch
from torch import nn

In [ ]:
import torchvision
from torchvision import transforms
from torchvision.transforms import ToTensor
from torchvision.datasets import FashionMNIST